# Example for General Setting 

Here, we show an example of how to use the package for general background velocities. Care must be taken that the geometric optics assumption still holds.

In [4]:
import numpy as np
import math 
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.sparse as sp

# Package functions hiding heavy lifting :)
from grt_inversion import layered_velocity, bilinear_velocity, one_velocity, extend_and_solve, generate_data_general, plot, compute_tau_and_a
from grt_inversion.reconstruction.translation_invariant.data_generation import n_func
from grt_inversion.reconstruction.translation_invariant import compute_ref_kernels, interpolate_kernels_fast_safe
from grt_inversion.utils import cutoff
from grt_inversion.reconstruction.general import compute_kernels_parallel_batched_general


SAVE = False
LOAD = False


In [ ]:
# Setup grid data for domain
nx, ny = 121, 121
a = 6 
x, y = np.linspace(-a, a, nx), np.linspace(0, 12, ny)

dx = (abs(x[1] - x[0]), abs(y[1] - y[0]))
alpha = 5

# Choose background velocity
# velocity = one_velocity
# velocity = layered_velocity
velocity = bilinear_velocity

# Setup for data space
t_data = np.linspace(15, 40, 701)
s_data = np.linspace(-10, 10, 111)

# Extend domain to include all source/receiver locations for s in s_data
ds = s_data[1] - s_data[0]
alpha_ind = math.ceil(alpha / ds)

s_data_extended = np.linspace(s_data[0] - alpha, s_data[-1] + alpha, len(s_data) + 2*alpha_ind)

# Solve eikonal and transport equations for all source locations in s_data_extended
tau_vals, a_vals, c = compute_tau_and_a(s_data_extended, x, y, velocity)

# Generate data
g = generate_data_general(t_data, dx, s_data_extended, s_data, c, alpha, tau_vals, a_vals, x_origin=x[0], func=n_func)

# Plot generated data
# plot(s_data, t_data, g)

In [ ]:
# Reconstruction parameters for GRT
s_vals = s_data # Source positions
t_vals = t_data  # Time samples


# Reconstruction parameters for reconstruction kernels
q, k = 2, 3 
gamma = .2

# Grid for values p where we determine the reconstruction
p1_vals, p2_vals = x, y
P1, P2 = np.meshgrid(p1_vals, p2_vals, indexing='ij')
p_vals = np.column_stack((P1.ravel(), P2.ravel()))

# Generate extended array
ds_vals = s_vals[1] - s_vals[0]
alpha_ind_vals = math.ceil(alpha / ds_vals)
s_vals_extended = np.linspace(s_vals[0] - alpha, s_vals[-1] + alpha, len(s_vals) + 2*alpha_ind_vals)

# Compute PDEs for reconstruction kernel grid
tau_vals, a_vals, c = compute_tau_and_a(s_vals_extended, x, y, velocity)

In [ ]:
# Compute kernel:

kernel = compute_kernels_parallel_batched_general(
    p_vals, gamma, q, k, t_vals, dx, s_vals_extended, s_vals, c, alpha, tau_vals, a_vals, x[0], batch_size=20
    )

# Reshape kernel:
ref_kernels= kernel.reshape(len(p1_vals), len(p2_vals), len(s_vals), len(t_vals))

In [9]:
if SAVE:
    # Save file
    filename = f"ref_kernel.npy"
    np.save(filename, ref_kernels)
    print(f"Saved to: {filename}")

if LOAD:
    # Load file
    ref_kernels = np.load('ref_kernels.npy')
    print("kernel loaded")


In [10]:
if np.any(s_vals != s_data) or np.any(t_vals != t_data):
    print("Interpolating kernels in parallel...")
    # Automatic selection based on problem size
    kernel_data = interpolate_kernels_fast_safe(ref_kernels, s_vals, t_vals, s_data, t_data, n_jobs=-1, method='auto')
    print("kernels interpolated")
else:
    print("No interpolation needed - using ref_kernels directly")
    kernel_data = ref_kernels

No interpolation needed - using ref_kernels directly


In [39]:
# Take inner products to compute approximate inversion
dt = t_data[1] - t_data[0]
ds = s_data[1] - s_data[0]
 
result = np.zeros((len(p1_vals), len(p2_vals)))

# Apply cutoff function
psi_g = sp.csr_matrix(cutoff(g, s_data, t_data))
# g = sp.csr_matrix(g)

for i in range(len(p1_vals)):
    for j in range(len(p2_vals)):
        K = sp.csr_matrix(kernel_data[i, j, :, :])
        #print(K.shape, g.shape)
        if sp.issparse(K):
            result[i, j] = ds * dt * np.sum(K.multiply(psi_g))
            #result[i, j] = ds * dt * np.sum(K.multiply(g))
        else:
            result[i, j] = ds * dt  * np.sum(K * psi_g)
            #result[i, j] = ds * dt  * np.sum(K * g)

In [ ]:
# Plot region of interest

# Filter the 1D coordinate arrays first
p1_mask = (p1_vals >= -5) & (p1_vals <= 5)
p2_mask = (p2_vals >= 1) & (p2_vals <= 9)

p1_sel = p1_vals[p1_mask]
p2_sel = p2_vals[p2_mask]

# Slice the result accordingly
result_sel = result[np.ix_(p1_mask, p2_mask)]

# Create meshgrid for plotting
P1, P2 = np.meshgrid(p1_sel, p2_sel, indexing='ij')

# Plot

with mpl.rc_context({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],  
    "axes.labelsize": 11,     
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "axes.titlesize": 11,
    "legend.fontsize": 9,
    "figure.dpi": 300,       
    "axes.linewidth": 0.8,    
    "lines.linewidth": 1.0,
    "lines.markersize": 4,    
    "text.latex.preamble": r"\usepackage{amsmath,amssymb}"
}):
    plt.figure(figsize=(4.5, 3.5))
    plt.pcolormesh(P1, P2, result_sel, shading='auto', cmap='Blues')
    plt.colorbar()
    plt.xlabel(r"$p_1$")
    plt.ylabel(r"$p_2$")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()